<a href="https://colab.research.google.com/github/Jeongmin-heo/tobigs_assignment/blob/master/week9_CNN_heojeongmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 2: Residual 모델 실행
- Colab을 활용하여 과제를 진행하도록 하겠습니다. 

In [0]:
cd /content/drive/My Drive/colab_workspace/ 

/content/drive/My Drive/colab_workspace


 ## model.py에서 작성한 forwad는 다음과 같습니다. 

Q. [TODO] 논문의 Figure 3 형태로 forward를 구성해주세요
  - 단, 논문의 구현과는 다른 형태이기 때문에 컨셉만 맞추어 구성하시면 됩니다.
  - layer1, residual_layers, global_avg, fc를 조합해주세요.


```

 def forward(self, x):
        x = self.layer1(x)
        x = self.residual_layers(x)
        x = self.global_avg(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
                
        return x
  ```     

In [0]:
# 과제를 진행하기 위한 압축파일을 풀었습니다. 
import zipfile as zf
files = zf.ZipFile("week9.zip","r")
files.extractall("diretory to extrat")
files.close


In [0]:
ls

 build/                 khaiii/     test/    week9/
'diretory to extrat'/   __MACOSX/   train/   week9.zip


In [0]:
cd /content/drive/My Drive/colab_workspace/week9 #디렉토리를 이동합니다. 

[Errno 2] No such file or directory: '/content/drive/My Drive/colab_workspace/week9 #디렉토리를 이동합니다.'
/content


In [0]:
ls

base/        data_loader/  model/           README.md  trainer/
config.json  LICENSE       parse_config.py  saved/     train.py
data/        logger/       __pycache__/     test.py    utils/


# config.json 

```
{
    "name": "CIFAR10_ResNet32",
    "n_gpu": 1,

    "arch": {
        "type": "ResNet32Model",
        "args": {"base_channels":64}
    },
    "data_loader": {
        "type": "CIFAR10DataLoader",
        "args":{
            "data_dir": "data/",
            "batch_size": 128,
            "shuffle": true,
            "validation_split": 0.1,
            "num_workers": 2
        }
    },
    "optimizer": {
        "type": "SGD",
        "args":{
            "lr": 0.1,
            "weight_decay": 0.0001
        }
    },
    "loss": "cross_entropy_loss",
    "metrics": [
        "accuracy", "top_k_acc"
    ],
    "lr_scheduler": {
        "type": "StepLR",
        "args": {
            "step_size": 50,
            "gamma": 0.1
        }
    },
    "trainer": {
        "epochs": 50,

        "save_dir": "saved/",
        "save_period": 1,
        "verbosity": 2,
        
        "monitor": "min val_loss",
        "early_stop": 10,

        "tensorboard": true
    }
}
```

### config.json 을 통해 사용할 기법과 파라미터를 조절할 수 있습니다. 
- 마치 ppt의 탬플릿처럼 연구자의 분석 방향과 방식에 따라 편리하게 변경할 수 있는 장점이 있습니다. 
- 참고링크 : https://github.com/victoresque/pytorch-template 


- train.py -c config.json를 진행하였습니다. 
- ResNet32Model을 사용하고 data_loader하여 0.1비율로 validation_split하였습니다. 
- Stochastic Gradient Descent을 사용하여 최적화를 진행하고 cross entropy loss를 사용하였습니다.
- 훈련 정보는 saved폴더에 저장하도록 하였습니다. 
- epochs는 50이지만 early_stop 조건을 추가하여 10번동안 유의미한 변화가 없을 경우 멈추도록 하겠습니다.


In [0]:
!python train.py -c config.json

Train Epoch: 10 [4224/45000 (9%)] Loss: 0.647548
Train Epoch: 10 [5632/45000 (13%)] Loss: 0.453673
Train Epoch: 10 [7040/45000 (16%)] Loss: 0.437876
Train Epoch: 10 [8448/45000 (19%)] Loss: 0.476556
Train Epoch: 10 [9856/45000 (22%)] Loss: 0.507134
Train Epoch: 10 [11264/45000 (25%)] Loss: 0.390866
Train Epoch: 10 [12672/45000 (28%)] Loss: 0.535822
Train Epoch: 10 [14080/45000 (31%)] Loss: 0.585607
Train Epoch: 10 [15488/45000 (34%)] Loss: 0.483016
Train Epoch: 10 [16896/45000 (38%)] Loss: 0.473991
Train Epoch: 10 [18304/45000 (41%)] Loss: 0.686128
Train Epoch: 10 [19712/45000 (44%)] Loss: 0.676894
Train Epoch: 10 [21120/45000 (47%)] Loss: 0.530834
Train Epoch: 10 [22528/45000 (50%)] Loss: 0.540940
Train Epoch: 10 [23936/45000 (53%)] Loss: 0.376120
Train Epoch: 10 [25344/45000 (56%)] Loss: 0.584408
Train Epoch: 10 [26752/45000 (59%)] Loss: 0.438758
Train Epoch: 10 [28160/45000 (63%)] Loss: 0.509332
Train Epoch: 10 [29568/45000 (66%)] Loss: 0.507020
Train Epoch: 10 [30976/45000 (69%)] L

- 나온정보를 토대로 "0328_091030"에 저장되어있는 모델이 best model임을 알 수 있었습니다. 
- 이를 토대로 test를 진행해보았습니다. 
**- <colab은 경로에 "경로"를 붙혀야 합니다>**

In [0]:
!python test.py --resume "/content/drive/My Drive/colab_workspace/week9/saved/models/CIFAR10_ResNet32/0328_091030/model_best.pth" 

Files already downloaded and verified
ResNet32Model(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (residual_layers): Sequential(
    (0): ResidualBlock(
      (residual_layer): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (1): ResidualBlock(
      (residual_layer): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU

- 'loss' = 0.8257985265731812
- 'accuracy' = 0.7246
- 'top_k_acc'=  0.9289

```
def top_k_acc(output, target, k=3):
    with torch.no_grad():
        pred = torch.topk(output, k, dim=1)[1]
        assert pred.shape[0] == len(target)
        correct = 0
        for i in range(k):
            correct += torch.sum(pred[:, i] == target).item()
    return correct / len(target)
```


- 일반적으로  top-1, top-5 acc를 많이 쓰는데 데이터의 클래스를 예측해서 확률값이 높은 k를 나열했을 때, k개 안에 진짜 정답이 포함되면 잘 예측한 것으로 판단하는 방식입니다.
- metric.py를 참고했을때 top_k_acc의 k는 **3**인 것을 알 수 있었습니다. 
- top 3를 예측했을때는 기존의 0.72보다 굉장히 높은 0.92로 나타는 것을 알 수 있었습니다. 


## tensorboard


In [0]:
!pip install tensorboard>=1.14.0

In [0]:
!pip install tensorboardX

In [0]:
!pip install 'git+https://github.com/lanpa/tensorboardX'

  Cloning https://github.com/lanpa/tensorboardX to /tmp/pip-req-build-ie38uwn9
  Running command git clone -q https://github.com/lanpa/tensorboardX /tmp/pip-req-build-ie38uwn9
  Created wheel for tensorboardX: filename=tensorboardX-2.0+d09aeb8-py2.py3-none-any.whl size=119726 sha256=b7ee19e217f0a6c3f7ce6bb7b239d236e368c5ee2984e1a57b3378dc26363968
  Stored in directory: /tmp/pip-ephem-wheel-cache-iizplw21/wheels/57/88/4d/30b1e9e2093375d22a0701b666d7e57f71d50cbdf8cc5998c4
Successfully built tensorboardX
  Found existing installation: tensorboardX 2.0
    Uninstalling tensorboardX-2.0:
      Successfully uninstalled tensorboardX-2.0


In [0]:
!tensorboard --logdir "/content/drive/My Drive/colab_workspace/week9/logger/visualization.py"

2020-03-28 19:35:06.328640: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Exception in thread Reloader:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/backend/application.py", line 586, in _reload
    multiplexer.AddRunsFromDirectory(path, name)
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/backend/event_processing/plugin_event_multiplexer.py", line 199, in AddRunsFromDirectory
    for subdir in io_wrapper.GetLogdirSubdirectories(path):
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/backend/event_processing/io_wrapper.py", line 181, in GetLogdirSubdirectories
    "GetLogdirSubdirectories: path exists and is not a " "directory, %s" % path
ValueError: GetLog